## making of a language model which will generate poem of four line
#### a second order marcov model will be used to store the probability of occurrence of three consecutive words/tokens
#### a first order marcov model will be used to store the probability of occurrence of each first two tokens in entire dataset
#### a normal one dim list to store prob of occurrence of first token of each line of entire dataset


In [1]:
import numpy as np
import pandas as pd
import string


In [2]:
np.random.seed(1234)

In [3]:
#  instead of storing probability of occurrence of three and two consecutive token in a matrix we will store in a dictionary
initial={}
firstOrder={}
secondOrder={}


In [4]:
# remove punctuation of each sentence
def remove_puncuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [5]:
# this funtion will put value V in corresponding key K  in specific dictionary D 
# this dictionary can be firstorder and secondOrder 
# K key can a single token or tupple of two token
def addToDict(D,K,V):
    if K not in D:
        D[K]=[]
    D[K].append(V)
    return

In [6]:
'''
this loop will extract the all robert frost poem from robert_frost.txt file
all the capital letter converted into small letter
initial_list:{} will contain count of occurrence of each first word of sentences

firstOrder:{} will contains  occurrence of each first two word of sentences. if first word is same then next word stored under 
                the same key which is the first word

secondOrder:{} will contains  occurrence of each three consecutive of words. if the first two word is same then all the third
                word all the third words will stored in a list 
            
''' 
for line in open('robert_frost.txt'):
    tokens=remove_puncuation(line.rstrip().lower()).split()
    length_token=len(tokens)
    for i in range(length_token):
        token_i=tokens[i]
        if i==0:
            initial[token_i]=initial.get(token_i,0)+1
        else:
            token_i_1=tokens[i-1]
            if i==length_token-1:
                addToDict(secondOrder,(token_i_1,token_i),'END')
            if i==1:
                addToDict(firstOrder,token_i_1,token_i)
            else:
                
                token_i_2=tokens[i-2]
                addToDict(secondOrder,(token_i_2,token_i_1),token_i)
    




In [7]:
# normalizing the distribution initial_prob
initial_prob={}
initial_total=sum(initial.values())
for k,v in initial.items():
    initial_prob[k]=initial[k]/initial_total


In [8]:
'''
in this fun a list of second word in first order and list of third word in second order dictionary will  be given 
as input. as a output a dictinary will be returned where each word is the key and values would be  corresponding count of
total occurance of that word/key in that input list 
'''
def listToPredict(toklist):
    pdict={}
    lengthoflist=len(toklist)
    for k in toklist:
        pdict[k]=pdict.get(k,0)+1
    for k,v in pdict.items():
        pdict[k]=v/lengthoflist
    return pdict
            

In [9]:
firstOrder_prob={}
for key, listvalue in firstOrder.items():
    firstOrder_prob[key]=listToPredict(listvalue)


In [10]:
secondOrder_prob={}
for key , listvalue in secondOrder.items():
    secondOrder_prob[key]=listToPredict(listvalue)

In [11]:
''' 
in this funtion a diction will be given as input 
a radom value generator will generate a rodom value between 0 to 1 and on the basis that random value 
and probability of each word in list , a word will be selected.
this basically help to generate each word in the poem
'''
def sample_word(D):
    p0=np.random.random()
    cumulative=0
    for t,p in D.items():
        cumulative+=p
        if p0<cumulative:
            return t
    assert(False)
    

In [19]:
'''
this funtion contains a loop which will go for four times 
in each loop first word be selected randomly  using initial_list[] 
on the basis of first word second word will be selected  using firstOrder{}
and after that using secondOrder{} each third word of the selected 
'''
def generate_poem():
    for i in range(4):
        sentence=[]
        
        w0 = sample_word(initial_prob)
        sentence.append(w0)
        w1= sample_word(firstOrder_prob[w0])
        sentence.append(w1)
        
        while(True):
            # print(secondOrder_prob[(w0,w1)])
            w2=sample_word(secondOrder_prob[(w0,w1)])
            if w2=="END":
                break
            sentence.append(w2)
            w0=w1
            w1=w2
        print(' '.join(sentence))

In [26]:
generate_poem()

and wheres john
or like the poetesss heart of one that didnt thrive
but where this old wall burns a sunny cheek
and swollen them
